In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2")
SOS_token = tokenizer.bos_token_id  # 使用 GPT2 的 <bos> 作为 SOS
EOS_token = tokenizer.eos_token_id  # 使用 GPT2 的 <eos> 作为 EOS
vocab_size = len(tokenizer)

# For English, lowercase, trim, and remove non-letter characters
def normalizeEng(s):
    '''
    process the English sentence
    '''
    s = s.lower().strip() # remove leading and trailing spaces then tokenize
    s = re.sub(r"([.!?])", r" \1", s) # add spaces around .!? to separate them from words
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s) # replace other characters with spaces
    return s.strip()
# For Chinese, trim, and remove other characters
def normalizeChi(s):
    s = s.strip()
    s = re.sub(r"([.!?。])", r" \1 ", s)
    s = re.sub(r"[^\u4e00-\u9fff\sa-zA-Z!！?。.]+", r" ", s)
    return s.strip()

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [3]:
def readLangs():
    '''
    read txt file, process sentence, convert them into pairs
    '''
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(dir, encoding='utf-8').read().strip().split('\n')

    # Split every line into chinese english pairs and normalize
    pairs = [[normalizeChi(l.split('\t')[1]),normalizeEng(l.split('\t')[0])] for l in lines]

    return pairs

In [4]:
def indexesFromSentence(sentence):
    return tokenizer(sentence)["input_ids"]

def tensorFromSentence(sentence):
    indexes = indexesFromSentence(sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(pair[0])
    target_tensor = tensorFromSentence(pair[1])
    return (input_tensor, target_tensor)

from torch.utils.data import random_split
def get_dataloader(batch_size, MAX_LENGTH=100):
    pairs = readLangs()

    n = len(pairs)
    input_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)
    target_ids = np.zeros((n, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(pairs):
        inp_ids = indexesFromSentence(inp)
        tgt_ids = indexesFromSentence(tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids[idx, :len(inp_ids)] = inp_ids
        target_ids[idx, :len(tgt_ids)] = tgt_ids

    data = TensorDataset(torch.LongTensor(input_ids).to(device),
                               torch.LongTensor(target_ids).to(device))

    train_size = int(0.9 * len(data))
    test_size = len(data) - train_size

    train_dataset, test_dataset = random_split(data, [train_size, test_size])

    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_dataloader, test_dataloader

In [5]:
import time

def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
                decoder_optimizer, criterion):
    total_loss = 0
    start_time = time.time()
    s_time = time.time()

    for i, data in enumerate(dataloader):
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

        if i % 100 == 0:
            print(f'Within one epoch: training {i}/{len(dataloader)} | Time: {time.time() - s_time:.2f}s')
            s_time = time.time()

    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'\nOne Epoch finished | Total time: {epoch_time:.2f}s | Average loss: {total_loss / len(dataloader):.4f}')

    return total_loss / len(dataloader)

## Encoder

In [6]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

## Decoder with attention

In [7]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

In [8]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

## Training

In [9]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [10]:
from tqdm import tqdm
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=1, plot_every=5):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in tqdm(range(1, n_epochs + 1),desc="Training Epochs..."):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

        if epoch % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
        print(f'==== Finished training of {epoch}/{n_epochs} epochs ====')

    showPlot(plot_losses)

In [11]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

## Evaluation

In [12]:
def tensor2sentence(input_tensor):
    return tokenizer.decode(input_tensor)
def evaluate(encoder, decoder, input_tensor):
    with torch.no_grad():
        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()[0]

        decoded_sentence = tensor2sentence(decoded_ids)
    return decoded_sentence, decoder_attn

In [13]:
def evaluateRandomly(dataloader,encoder, decoder):
    for data in dataloader:
        input_tensor,target_tensor = data
        print('>', tensor2sentence(input_tensor[0]).split('<|endoftext|>', 1)[0])
        print('=', tensor2sentence(target_tensor[0]).split('<|endoftext|>', 1)[0])
        output_sentence, _ = evaluate(encoder, decoder, input_tensor)
        print('<', output_sentence.split('<|endoftext|>', 1)[0])
        print('')

# main

In [14]:
hidden_size = 128
batch_size = 32
dir = '/kaggle/input/cmn-eng/cmn.txt'
MAX_LENGTH = 100

train_dataloader, test_dataloader = get_dataloader(batch_size)

encoder = EncoderRNN(vocab_size, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, vocab_size).to(device)

train(train_dataloader, encoder, decoder, 30, print_every=5, plot_every=5)

Reading lines...


Training Epochs...:   0%|          | 0/30 [00:00<?, ?it/s]

Within one epoch: training 0/842 | Time: 1.11s
Within one epoch: training 100/842 | Time: 25.40s
Within one epoch: training 200/842 | Time: 25.55s
Within one epoch: training 300/842 | Time: 25.64s
Within one epoch: training 400/842 | Time: 25.59s
Within one epoch: training 500/842 | Time: 25.58s
Within one epoch: training 600/842 | Time: 25.59s
Within one epoch: training 700/842 | Time: 25.60s
Within one epoch: training 800/842 | Time: 25.58s


Training Epochs...:   3%|▎         | 1/30 [03:36<1:44:25, 216.06s/it]


One Epoch finished | Total time: 216.06s | Average loss: 0.5787
==== Finished training of 1/30 epochs ====
Within one epoch: training 0/842 | Time: 0.26s
Within one epoch: training 100/842 | Time: 25.52s
Within one epoch: training 200/842 | Time: 25.60s
Within one epoch: training 300/842 | Time: 25.47s
Within one epoch: training 400/842 | Time: 25.58s
Within one epoch: training 500/842 | Time: 25.49s
Within one epoch: training 600/842 | Time: 25.55s
Within one epoch: training 700/842 | Time: 25.51s
Within one epoch: training 800/842 | Time: 25.70s


...


One Epoch finished | Total time: 215.41s | Average loss: 0.0824
==== Finished training of 28/30 epochs ====
Within one epoch: training 0/842 | Time: 0.26s
Within one epoch: training 100/842 | Time: 25.57s
Within one epoch: training 200/842 | Time: 25.50s
Within one epoch: training 300/842 | Time: 25.56s
Within one epoch: training 400/842 | Time: 25.52s
Within one epoch: training 500/842 | Time: 25.53s
Within one epoch: training 600/842 | Time: 25.71s
Within one epoch: training 700/842 | Time: 25.57s
Within one epoch: training 800/842 | Time: 25.51s


Training Epochs...:  97%|█████████▋| 29/30 [1:44:08<03:35, 215.35s/it]


One Epoch finished | Total time: 215.15s | Average loss: 0.0803
==== Finished training of 29/30 epochs ====
Within one epoch: training 0/842 | Time: 0.26s
Within one epoch: training 100/842 | Time: 25.48s
Within one epoch: training 200/842 | Time: 25.55s
Within one epoch: training 300/842 | Time: 25.60s
Within one epoch: training 400/842 | Time: 25.55s
Within one epoch: training 500/842 | Time: 25.60s
Within one epoch: training 600/842 | Time: 25.53s
Within one epoch: training 700/842 | Time: 25.65s
Within one epoch: training 800/842 | Time: 25.64s


Training Epochs...: 100%|██████████| 30/30 [1:47:43<00:00, 215.46s/it]


One Epoch finished | Total time: 215.29s | Average loss: 0.0782
107m 48s (- 0m 0s) (30 100%) 0.0825
==== Finished training of 30/30 epochs ====


In [15]:
# 保存模型
encoder_path = 'encoder_model.pth'
decoder_path = 'decoder_model.pth'

torch.save(encoder.state_dict(), encoder_path)
torch.save(decoder.state_dict(), decoder_path)

print(f"Encoder model saved to {encoder_path}")
print(f"Decoder model saved to {decoder_path}")

Encoder model saved to encoder_model.pth
Decoder model saved to decoder_model.pth


In [16]:
# evaluation
encoder.eval()
decoder.eval()
evaluateRandomly(test_dataloader,encoder, decoder)

> 老鼠是胆小的生物 。
= a mouse is a timid creature
< the old man is a child

> 你买了往返的票吗
= did you buy a round trip ticket ?
< did you ski this sewing ?

> 你能在放學後幫助我們嗎
= could you help us after school ?
< can you teach me how to meet us any other birthday ?

> 我不喜欢茶 所以我早餐大致上都喝咖啡 。
= i do not like tea so i generally drink coffee for breakfast
< i don t like any other big morning with them too

> 湯姆看來總是在工作 。
= tom always seems to be working
< tom seems to be a job

> 他還沒有真正去過美國 。
= he hasn t actually been to america
< he didn t sleep on the weather

> 她被要求去说服他以让他或者他的儿子或者是别的人来粉刷屋子 。
= she was asked to convince him to get his son or someone else to paint the house
< she was considered to his parents he grows in the world

> 医生在观察X光片 。
= the doctors are looking at an x ray
< the doctor was born in the city

> 羅馬這個城市值得一遊 。
= rome is a city worth visiting
< the picture is worth a sport from wood

> 你覺得累嗎
= are you feeling tired ?
< do you feel tired ?

> 除了雨天我每天都散步 。
= i take a walk every day except w